In [ ]:
!sudo apt-get update && apt-get install -y libspatialindex-dev

In [ ]:
!pip install rtree pygeos geopandas descartes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from tabulate import tabulate
%matplotlib inline  

In [ ]:
df = pd.read_csv("data.csv", index_col=0)
df = df.sort_values('y')
# df = df.drop(columns="geometry")
df.head()


In [ ]:
import geopandas
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.x, df.y), 
                            crs="+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")
gdf.head()

In [ ]:
gdf = gdf.drop(columns=['x', 'y'])
gdf.tail()

In [ ]:
gdf.plot(markersize=.1, figsize=(8, 8))

In [ ]:
ax = gdf.plot(markersize=.1, figsize=(12, 8), column='dob', cmap='jet')
plt.autoscale(False)
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.to_crs(gdf.crs).plot(ax=ax, color='none', edgecolor='black')
ax.axis('off')

In [ ]:
xmin, ymin, xmax, ymax= gdf.total_bounds

n_cells=30
cell_size = (xmax-xmin)/n_cells

crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"

grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
cell = geopandas.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs=crs)


In [ ]:
ax = gdf.plot(markersize=.1, figsize=(12, 8), column='dob', cmap='jet')
plt.autoscale(False)
cell.plot(ax=ax, facecolor="none", edgecolor='grey')
ax.axis("off")

In [ ]:
merged = geopandas.sjoin(gdf, cell, how='left', op='within')
# make a simple count variable that we can sum
merged['n_fires']=1
# Compute stats per grid cell -- aggregate fires to grid cells with dissolve
dissolve = merged.dissolve(by="index_right", aggfunc="count")

In [ ]:
cell.loc[dissolve.index, 'n_fires']=dissolve.n_fires.values

In [ ]:
dissolve.to_crs(cell.crs)

In [ ]:
ax = cell.plot(column='n_fires', figsize=(12, 8), cmap='viridis', vmax=5000, edgecolor="grey")
plt.autoscale(False)
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.to_crs(cell.crs).plot(ax=ax, color='none', edgecolor='black')
ax.axis('off')

In [ ]:
merged.to_file("grids.geojson", driver='GeoJSON')
dissolve.to_file("dissolved_grids.geojson", driver='GeoJSON')
world.to_file("world_grids.geojson", driver='GeoJSON')
cell.to_file("cell_grids.geojson", driver='GeoJSON')


In [ ]:
cell.to_crs(cell.crs)


In [ ]:
world.to_crs(cell.crs).plot(ax=ax, color='none', edgecolor='black')

In [ ]:
df = pd.read_csv("cell_grids.csv")

from shapely import wkt

df['geometry'] = df['geometry'].apply(wkt.loads)
# gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
gdf = geopandas.GeoDataFrame(df,
                            crs="+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")
gdf.to_file("cell_grids_colors.geojson", driver="GeoJSON")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cbdba8d8-bec8-4acf-b7f8-c64fc2312409' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>